# 06-02

- Transition Matrix only
- Euclidean Distance

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, loadCoverGroups
import numpy as np
from sklearn.preprocessing import StandardScaler

from ipynb.fs.full.SearchInfo import searchByID, searchBySpotify, getTrackSpotifyInfo
from ipynb.fs.full.Utils import append_df_to_csv,transformToPercentage

In [2]:
cover = loadCoverGroups()
listCovers = cover[0]
originalSongId = listCovers[0]
coverSongId = listCovers[1]

In [3]:
cover

[[180849,
  6001027,
  1686718,
  5000013,
  5000001,
  5000007,
  5000008,
  5000023,
  5000017,
  5000019,
  5000020,
  5000018,
  5000011,
  5000012],
 [6001971, 6001984],
 [6002492, 6002491],
 [6000699, 6001165, 6000182],
 [6000104, 6000273],
 [6000768, 6000784],
 [6001931, 6001748],
 [6002043, 6002073],
 [6002244, 6000194],
 [6001209, 6000606],
 [6000352, 6001248],
 [6000549, 6001508, 6001242],
 [6001918, 6001997],
 [6001651, 6001900],
 [6000263, 6001570, 6002374],
 [6001822, 6001653],
 [6000763, 6000668],
 [6000764, 6000667],
 [6001543, 6001541],
 [6000762, 6000773],
 [6000771, 6000670],
 [6001747, 6001952],
 [6000548, 6002203],
 [6000172, 6002133],
 [6001762, 6001675],
 [6002147, 6002146],
 [6000767, 6000775],
 [6000766, 6000673],
 [6001506, 6000529],
 [6000397, 6000409],
 [6001914, 6002001],
 [6002482, 6002454],
 [6000765, 6000783],
 [6000761, 6000782],
 [6000814, 6000813],
 [6001224, 6000140]]

In [4]:
spotifyFeatures, transitionMatrices, df = loadDataFrames()

In [5]:
transitionMatrices

,0,1,2,3,4,5,6,7,8,9,...,1284,1286,1287,1288,1290,1291,1292,1293,1294,1295
id,,,,,,,,,,,,,,,,,,,,,
23,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
105,0.185185,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
118,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.044944,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
120,0.029412,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
292,0.013889,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.013889,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002496,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
6002497,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0
6002498,0.000000,0.015385,0.0,0.015385,0.0,0.015385,0.030769,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0


In [6]:
transitionMatrices.sum(axis = 1).sum(axis=0)

5979.999999999996

## Applying eucledian distance

In [7]:
def euclidean_distance(x, y):   
    return np.sqrt(np.sum((x - y) ** 2))

In [8]:
originalSongVector = transitionMatrices.loc[[originalSongId]].iloc[0]

***Dropping the cover song***

In [9]:
ranking = []
for index in range(0, len(transitionMatrices)):
    ranking.append([transitionMatrices.index[index], euclidean_distance(transitionMatrices.iloc[index, :], originalSongVector)])

In [10]:
ranksDF = pd.DataFrame(ranking)

In [11]:
ranksDF = ranksDF.rename(columns={0: "id", 1: "rank"})

In [12]:
ranksDF

,id,rank
0,23,0.460357
1,105,0.386437
2,118,0.476548
3,120,0.547385
4,292,0.416708
...,...,...
5975,6002496,0.381281
5976,6002497,0.532379
5977,6002498,0.422343
5978,6002499,0.431668


In [13]:
ranksDF = ranksDF.sort_values(by="rank")

In [14]:
ranksDF.reset_index(drop=True, inplace=True)

In [15]:
ranksDF.head(10)

,id,rank
0,180849,0.000000
1,2846348,0.158068
2,1186397,0.176593
3,1148352,0.177127
4,1168573,0.185495
5,1909308,0.197408
6,2065851,0.198064
7,532785,0.200606
8,1106926,0.207606
9,41961,0.208309


In [16]:
ranksDF[ranksDF['id']==coverSongId]

,id,rank
161,6001027,0.308294


## Rank for original Song 156

In [17]:
ranksDF.loc[ranksDF.id.isin(listCovers)]

,id,rank
0,180849,0.000000
28,5000007,0.238091
51,5000019,0.263030
56,5000018,0.267234
138,5000020,0.301116
161,6001027,0.308294
173,5000008,0.310970
189,1686718,0.314095
260,5000011,0.325204
271,5000012,0.326780


In [19]:
# To extract covers info change path in util if needed
ranks = transformToPercentage(ranksDF.head(10), '0200')
ranks

,id_chord,id_spotify_track,name,release_date,percentage,rank,position,id_experiment
0,180849,3jButwtJMLx3Ub61BfRiHh,the scientist,2022-06-03,100.000,0.000000,0,0200
1,2846348,4j5ffIFh7bFT7GZciP1TCy,10000 hours,2021-08-13,99.842,0.158068,1,0200
2,1186397,01uqI4H13Gsd8Lyl1EYd8H,same love,2012-10-09,99.823,0.176593,2,0200
3,1148352,2R6UrJ8uWbSIiHWmvRQvN8,whiskey in the jar,1998-01-01,99.823,0.177127,3,0200
4,1168573,1NNAI51EuoRWw1ydX1zV7S,daylight,2012-01-01,99.815,0.185495,4,0200
5,1909308,35r28RDot7nPE7y9K9H7l0,feeling whitney,2016-12-09,99.803,0.197408,5,0200
6,2065851,43IbCqMcMvZi1v16LFAE3n,safe,2017-06-01,99.802,0.198064,6,0200
7,532785,6Hmj7SrLRbreLVfVS7mV1S,wake up,2004-06-16,99.799,0.200606,7,0200
8,1106926,78meKU7RcdstGohFRaQUMF,ghosts that we knew,2012-09-21,99.792,0.207606,8,0200
9,41961,1JkZg3eMQTmTn93E8Yd3UL,i want you to want me,1977-09,99.792,0.208309,9,0200
